# Notebook for the testing on the Teresa Robot
## Importing all necessary tools

In [1]:
from src.gym_envs.RobotEnv import RobotEnv # Training environment
import roslibpy # API of ROS
from src.utils.training_tools import NB_STATES

import os
import sys
import gym
from random import random, randrange, sample
import numpy as np

from tqdm import tqdm
from collections import deque
import pandas as pd

import keras.backend as K
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Reshape, LSTM, Lambda
from keras.regularizers import l2
from keras.utils import to_categorical

import tensorflow as tf
import tensorflow
from keras.layers import Conv2D, MaxPooling2D
from src.robots.Teresa_adap import Teresa # This is the representation of Teresa Robot
from src.utils.training_tools import NB_STATES

from src.robots.actions.camera_adap import DlinkDCSCamera # class for the camera


:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named 'service_identity''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


## Creating the connection with ROS

In [2]:
#ip_ordi : 192.168.1.50

client = roslibpy.Ros(host="192.168.1.14", port=9090)

client.run()
print(client.is_connected)

True


## Creating the connection with the camera

In [3]:
host='192.168.1.35'
user='admin'
password='123456'
camera = DlinkDCSCamera(host = host, user = user, password = password)
camera.set_day_night(2)

{'DayNightMode': '2',
 'LightSensorControl': '3',
 'IRLedScheduleSunStart': '00:00',
 'IRLedScheduleSunEnd': '00:00',
 'IRLedScheduleMonStart': '00:00',
 'IRLedScheduleMonEnd': '00:00',
 'IRLedScheduleTueStart': '00:00',
 'IRLedScheduleTueEnd': '00:00',
 'IRLedScheduleWedStart': '00:00',
 'IRLedScheduleWedEnd': '00:00',
 'IRLedScheduleThuStart': '00:00',
 'IRLedScheduleThuEnd': '00:00',
 'IRLedScheduleFriStart': '00:00',
 'IRLedScheduleFriEnd': '00:00',
 'IRLedScheduleSatStart': '00:00',
 'IRLedScheduleSatEnd': '00:00'}

## Testing the Neural Network

### Necessary functions for the training (Neural Network Set up)

In [4]:

class SumTree:
    write = 0

    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros( 2*capacity - 1 )
        self.data = np.zeros( capacity, dtype=object )

    def _propagate(self, idx, change):
        parent = (idx - 1) // 2

        self.tree[parent] += change

        if parent != 0:
            self._propagate(parent, change)

    def _retrieve(self, idx, s):
        left = 2 * idx + 1
        right = left + 1

        if left >= len(self.tree):
            return idx

        if s <= self.tree[left]:
            return self._retrieve(left, s)
        else:
            return self._retrieve(right, s-self.tree[left])

    def total(self):
        return self.tree[0]

    def add(self, p, data):
        idx = self.write + self.capacity - 1

        self.data[self.write] = data
        self.update(idx, p)

        self.write += 1
        if self.write >= self.capacity:
            self.write = 0

    def update(self, idx, p):
        change = p - self.tree[idx]

        self.tree[idx] = p
        self._propagate(idx, change)

    def get(self, s):
        idx = self._retrieve(0, s)
        dataIdx = idx - self.capacity + 1

        return (idx, self.tree[idx], self.data[dataIdx])

class MemoryBuffer(object):
    """ Memory Buffer Helper class for Experience Replay
    using a Sum Tree (for PER)
    """
    def __init__(self, buffer_size):
        """ Initialization
        """
        
        # Prioritized Experience Replay
        self.alpha = 0.6
        self.epsilon = 0.01
        self.buffer = SumTree(buffer_size)

        self.count = 0
        self.buffer_size = buffer_size

    def memorize(self, state, action, reward, done, new_state, error=None):
        """ Save an experience to memory, optionally with its TD-Error
        """

        experience = (state, action, reward, done, new_state)
        priority = self.priority(error[0])
        self.buffer.add(priority, experience)
        self.count += 1


    def priority(self, error):
        """ Compute an experience priority, as per Schaul et al.
        """
        return (error + self.epsilon) ** self.alpha

    def size(self):
        """ Current Buffer Occupation
        """
        return self.count

    def sample_batch(self, batch_size):
        """ Sample a batch, optionally with (PER)
        """
        batch = []

    
        T = self.buffer.total() // batch_size
        for i in range(batch_size):
            a, b = T * i, T * (i + 1)
            s = np.random.uniform(a, b)
            idx, error, data = self.buffer.get(s)
            batch.append((*data, idx))
        idx = np.array([i[5] for i in batch])

        # Return a batch of experience
        s_batch = np.array([i[0] for i in batch])
        a_batch = np.array([i[1] for i in batch])
        r_batch = np.array([i[2] for i in batch])
        d_batch = np.array([i[3] for i in batch])
        new_s_batch = np.array([i[4] for i in batch])
        return s_batch, a_batch, r_batch, d_batch, new_s_batch, idx

    def update(self, idx, new_error):
        """ Update priority for idx (PER)
        """
        self.buffer.update(idx, self.priority(new_error))

    def clear(self):
        """ Clear buffer / Sum Tree
        """
        self.buffer = SumTree(buffer_size)
        self.count = 0

In [5]:
def conv_layer(d, k):
    """ Returns a 2D Conv layer, with and ReLU activation
    """
    return Conv2D(d, k, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')

def conv_block(inp, d=3, pool_size=(2, 2), k=3):
    """ Returns a 2D Conv block, with a convolutional layer, max-pooling
    """
    conv = conv_layer(d, k)(inp)
    return MaxPooling2D(pool_size=pool_size)(conv)

class Agent:
    """ Agent Class (Network) for DDQN
    """

    def __init__(self, state_dim, action_dim, lr, tau):
        
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.tau = tau
        
        # Initialize Deep Q-Network
        self.model = self.network()
        self.model.compile(Adam(lr), 'mse')
        
        # Build target Q-Network
        self.target_model = self.network()
        self.target_model.compile(Adam(lr), 'mse')
        self.target_model.set_weights(self.model.get_weights())

    def huber_loss(self, y_true, y_pred):
        return K.mean(K.sqrt(1 + K.square(y_pred - y_true)) - 1, axis=-1)

    def network(self):
        """ Build Deep Q-Network
        """
        inp = Input((self.state_dim))

        # Determine whether we are dealing with an image input or not
        if(len(self.state_dim) > 2):
            inp = Input((self.state_dim[1:]))
            x = conv_block(inp, 32, (2, 2), 8)
            x = conv_block(x, 64, (2, 2), 4)
            x = conv_block(x, 64, (2, 2), 3)
            x = Flatten()(x)
            x = Dense(256, activation='relu')(x)
        else:
            x = Flatten()(inp)
            x = Dense(64, activation='relu')(x)
            x = Dense(64, activation='relu')(x)


        # Have the network estimate the Advantage function as an intermediate layer
        x = Dense(self.action_dim + 1, activation='linear')(x)
        x = Lambda(lambda i: K.expand_dims(i[:,0],-1) + i[:,1:] - K.mean(i[:,1:], keepdims=True), output_shape=(self.action_dim,))(x)

        return Model(inp, x)

    def transfer_weights(self):
        """ Transfer Weights from Model to Target at rate Tau
        """
        W = self.model.get_weights()
        tgt_W = self.target_model.get_weights()
        for i in range(len(W)):
            tgt_W[i] = self.tau * W[i] + (1 - self.tau) * tgt_W[i]
        self.target_model.set_weights(tgt_W)

    def fit(self, inp, targ):
        """ Perform one epoch of training
        """
        self.model.fit(self.reshape(inp), targ, epochs=1, verbose=0)

    def predict(self, inp):
        """ Q-Value Prediction
        """
        inp = np.array(inp)
        return self.model.predict(self.reshape(inp))

    def target_predict(self, inp):
        """ Q-Value Prediction (using target network)
        """
        inp = np.array(inp)
        return self.target_model.predict(self.reshape(inp))

    def reshape(self, x):
        if len(x.shape) < 4 and len(self.state_dim) > 2: return np.expand_dims(x, axis=0)
        elif len(x.shape) < 3: return np.expand_dims(x, axis=0)
        else: return x

    def Save(self, path):
        self.model.save_weights(path + '.h5')
        self.model.save(path)

    def load_weights(self, path):
        #self.model.load_weights(path)
        self.model = tf.keras.models.load_model(path)

In [6]:
class DDQN:
    """ Deep Q-Learning Main Algorithm
    """

    def __init__(self, action_dim, state_dim):
        """ Initialization
        """
        # Environment and DDQN parameters
        self.action_dim = action_dim
        self.state_dim = state_dim
        self.lr = 0.01
        self.gamma = 0.95
        self.epsilon = 0.8
        self.epsilon_decay = 0.99
        self.buffer_size = 20000
        
        #
        if(len(self.state_dim) < 3):
            self.tau = 1e-2
        else:
            self.tau = 1.0
            
        # Create actor and critic networks
        self.agent = Agent(self.state_dim, action_dim, self.lr, self.tau)
        
        # Memory Buffer for Experience Replay
        self.buffer = MemoryBuffer(self.buffer_size)

    def policy_action(self, s):
        """ Apply an espilon-greedy policy to pick next action
        """
        if random() <= self.epsilon:
            return randrange(self.action_dim)
        else:
            return np.argmax(self.agent.predict(s)[0])

    def action(self, s):
        """ Apply an espilon-greedy policy to pick next action
        """
        return np.argmax(self.agent.predict(s)[0])

    def train_agent(self, batch_size):
        """ Train Q-network on batch sampled from the buffer
        """
        # Sample experience from memory buffer with PER
        s, a, r, d, new_s, idx = self.buffer.sample_batch(batch_size)

        # Apply Bellman Equation on batch samples to train our DDQN
        q = self.agent.predict(s)
        next_q = self.agent.predict(new_s)
        q_targ = self.agent.target_predict(new_s)

        for i in range(s.shape[0]):
            old_q = q[i, a[i]]
            if d[i]:
                q[i, a[i]] = r[i]
            else:
                next_best_action = np.argmax(next_q[i,:])
                q[i, a[i]] = r[i] + self.gamma * q_targ[i, next_best_action]
           
            self.buffer.update(idx[i], abs(old_q - q[i, a[i]]))
                
        # Train on batch
        self.agent.fit(s, q)
        # Decay epsilon
        self.epsilon *= self.epsilon_decay


    def train(self, env, nb_episodes, batch_size):
        """ Main DDQN Training Algorithm
        """

        results = []
        for e in range(nb_episodes):
            
            # Reset episode
            time,cumul_reward, done  = 1,0, False
            old_state = env.reset()

            while not done:
                
                # Actor picks an action (following the policy)
                a = self.policy_action(old_state)
                
                # Retrieve new state, reward, and whether the state is terminal
                #on a changé ici avec quatre actions
                #print("l et a", a)
                new_state, r, done = env.step(a, time)
                
                #render the drone image
                env.render()
                
                # Memorize for experience replay
                self.memorize(old_state, a, r, done, new_state)
                
                # Update current state
                old_state = new_state
                cumul_reward += r
                
                # Train DDQN and transfer weights to target network
                if(self.buffer.size() > batch_size):
                    self.train_agent(batch_size)
                    self.agent.transfer_weights()
                
                # Time limit to put the target in the center
                time += 1
                if(time >= 10):
                    done = True
                    
                

            # Display score
            print(str(e)+  "/"+str(nb_episodes)+"  Score: " + str(cumul_reward))
            if (e%20 == 0):self.Save_weights('Model/new_modelo')

        return results

    def memorize(self, state, action, reward, done, new_state):
        """ Store experience in memory buffer
        """

        q_val = self.agent.predict(state)
        q_val_t = self.agent.target_predict(new_state)
        next_best_action = np.argmax(self.agent.predict(new_state))
        new_val = reward + self.gamma * q_val_t[0, next_best_action]
        td_error = abs(new_val - q_val)[0]

        self.buffer.memorize(state, action, reward, done, new_state, td_error)

    def Save_weights(self, path):
        self.agent.Save(path)

    def load_weights(self, path):
        self.agent.load_weights(path)

## Testing the Neural Network

In [ ]:
def main():

    client.run() # This run the main loop of ROS
    teresa_controller = Teresa(client) # Robot API
    env = RobotEnv(teresa_controller, client) # Training Environment

    #env.reset()
    state_dim = (1,3)
    action_dim = 4

    # Pick algorithm to train
    net = DDQN(action_dim, state_dim)
    net.load_weights('Final_Model/new_modelo')
    
    env.reset()
    old_state, r, done = env.step(0,1)
    i = 0
    
   
    while i < 1:
        time_=0
        while not done:
            a = net.action(old_state) #changed
            old_state, r, done = env.step(a,1)
            env.render()
            env.render()
            # Time limit to put the target in the center
            time_ += 1
            if(time_ >= 10):
                done = True
        while done:
            old_state, r, done = env.step(0,1)
            env.render()
        i += 1
          

if __name__ == "__main__":
    main()

{'angular': {'y': 0.2, 'x': 0.2, 'z': 0.2}}
{'linear': {'x': -0.5}}
{'linear': {'x': -0.5}}
{'linear': {'x': -0.5}}
{'linear': {'x': 0.5}}
{'angular': {'y': 0.2, 'x': 0.2, 'z': 0.2}}
{'linear': {'x': 0.5}}
